Installing dependencies

In [1]:
!pip install PyPDF2 scikit-learn pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00


Importing Libraries

In [2]:
import PyPDF2
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re


Dataset (Sample job/resume classification data)

In [8]:
data = {
    "resume_text": [
        "Python, Data analysis, Machine Learning, Pandas, SQL, Statistics",
        "Java, Spring Boot, Microservices, REST API, AWS",
        "Excel, Financial reporting, Forecasting, Accounting, Power BI",
        "Deep learning, CNN, image processing, TensorFlow, Keras",
        "Customer support, communication skills, sales CRM"
    ],
    "label": ["Data Science", "Software", "Finance", "AI Engineer", "Support"]
}

df = pd.DataFrame(data)
df


,resume_text,label
0,"Python, Data analysis, Machine Learning, Panda...",Data Science
1,"Java, Spring Boot, Microservices, REST API, AWS",Software
2,"Excel, Financial reporting, Forecasting, Accou...",Finance
3,"Deep learning, CNN, image processing, TensorFl...",AI Engineer
4,"Customer support, communication skills, sales CRM",Support


from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('resume_text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['label'].value_counts()
    for x_label, grp in df.groupby('resume_text')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('resume_text')
_ = plt.ylabel('label')

Converting to TF-IDF

In [4]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df["resume_text"])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


Accuracy: 0.0
              precision    recall  f1-score   support

    Software       0.00      0.00      0.00       1.0
     Support       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

Resume PDF Reader Function

In [5]:
def extract_text_from_pdf(uploaded_file):
    pdf_reader = PyPDF2.PdfReader(uploaded_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text


Skill Extraction Function

In [6]:
skills_list = ["python", "sql", "excel", "machine learning", "deep learning", "power bi",
               "nlp", "aws", "java", "tableau", "statistics", "pandas", "r", "communication"]

def extract_skills(text):
    text = text.lower()
    found = []
    for skill in skills_list:
        if skill in text:
            found.append(skill)
    return list(set(found))


Uploading & Predicting Resume Category

In [7]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    resume_text = extract_text_from_pdf(filename)
    cleaned_text = re.sub(r'\s+', ' ', resume_text)
    vectorized = tfidf.transform([cleaned_text])
    prediction = model.predict(vectorized)[0]

    print("\n Resume Name:", filename)
    print(" Predicted Job Role:", prediction)
    print(" Extracted Skills:", extract_skills(cleaned_text))


Saving AZHAR DETAILED RESUME.pdf to AZHAR DETAILED RESUME.pdf

 Resume Name: AZHAR DETAILED RESUME.pdf
 Predicted Job Role: Data Science
 Extracted Skills: ['sql', 'r', 'deep learning', 'power bi', 'machine learning', 'nlp', 'excel', 'statistics', 'python']
